# Deep RL Assignment 1: Imitation Learning
## Prepare

In [1]:
# Import dependencies
import sys,os
sys.path.append(os.path.abspath('..'))
import tensorflow as tf
import numpy as np
import tf_util
import gym

home_work_root=os.path.abspath('..')


def collect_expert_data(env_name, max_timesteps=None, num_rollouts=20, render=False):
    from load_policy import load_policy
    policy_fn = load_policy(os.path.join(home_work_root, 'experts', env_name+'.pkl'))
    
    with tf.Session():
        tf_util.initialize()
        env = gym.make(env_name)
        max_steps = max_timesteps or env.spec.timestep_limit

        returns = []
        observations = []
        actions = []
        for i in range(num_rollouts):
            obs = env.reset()
            done = False
            totalr = 0.
            steps = 0
            while not done:
                action = policy_fn(obs[None,:])
                observations.append(obs)
                actions.append(action)
                obs, r, done, _ = env.step(action)
                totalr += r
                steps += 1
                if render:
                    env.render()
                if steps % 10 == 0:
                    print("\rIter{0}/{1}: {2}/{3}".format(i+1, num_rollouts, steps+1, max_steps), end='')
                if steps >= max_steps:
                    break
            returns.append(totalr)
        print('. done')

        #print('returns', returns)
        print('mean return', np.mean(returns))
        print('std of return', np.std(returns))
    return {'observation': observations, 'actions': actions}

def test_policy(env_name, policy, max_timesteps=None, num_rollouts=20, render=False):
    return None

In [2]:
env_names=['Humanoid-v1',]
expert_data={env: collect_expert_data(env, num_rollouts=20) for env in env_names}

[2017-07-04 22:15:16,186] Making new env: Humanoid-v1


Iter20/20: 1001/1000. done
mean return 10412.4788075
std of return 62.6522079497


In [ ]:
def build_model(env_name):
    pass

for env_name in expert_data:
    obs = expert_data[env_name]['observation']
    actions = expert_data[env_name]['actions']
    y_, x_,  = build_model(env_name)